In [1]:
using Dates
using BenchmarkTools
using Fastback

In [16]:
function run_backtest()
    # create instrument
    inst = Instrument(1, "AAPL")
    insts = [inst]

    # market data (order books)
    data = MarketData(insts)
    book = data.order_books[inst.index]

    # create trading account
    acc = Account(insts, 100_000.0)

    dt = DateTime(2018, 1, 2, 9, 30, 0)
    for i = 1:1_000_000
        ba = BidAsk(dt, 100.0 + i / 1000, 102 + i / 1000)
        update_book!(book, ba)
        update_account!(acc, data, inst)
        execute_order!(acc, book, Order(inst, 1.0, dt))
        execute_order!(acc, book, Order(inst, -1.0, dt))
    end
end;

In [18]:
# 328 ms (Memory estimate: 425.63 MiB, allocs estimate: 2000037.)
@benchmark run_backtest() evals=1 samples=50

BenchmarkTools.Trial: 40 samples with 1 evaluation.
 Range (min … max):  107.200 ms … 144.936 ms  ┊ GC (min … max):  0.00% … 25.74%
 Time  (median):     125.522 ms               ┊ GC (median):    13.30%
 Time  (mean ± σ):   126.218 ms ±   8.587 ms  ┊ GC (mean ± σ):  13.98% ±  5.55%

                       ▅█                   ▂                    
  ▅▁▁▁▁▁▁▅▁▅▅▁▁▁▁▅▁▁▅▁▅██▅▁▁▅▅▁▅▅▁█▅▅▁▅▅▁▁█▅█▁▁▅█▁▁█▁▁▁▁▁▁▁▁▅▁▅ ▁
  107 ms           Histogram: frequency by time          145 ms <

 Memory estimate: 212.65 MiB, allocs estimate: 1000035.

In [4]:
# using ProfileView
# ProfileView.@profview map(i -> run_backtest(), 1:100)